In [ ]:
import os
import gzip
import sqlite3
import zlib
import io
from sqlite3 import Error
import pandas as pd
from collections import OrderedDict

from Bio import SeqIO 
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

# SQNce Mapping-related Tables

In [ ]:
# TODO add documentation to all SQNce creation functions

# Establish connection with SQNce.db, generating a new SQLite3 database if needed
def sql_connection():
    try:
        con = sqlite3.connect('SQNce.db')
        print("Connection established.")
        return(con)
    except Error:
        print(Error)

# After establishing connection with SQNce create the specified tables
def sql_table(con):
    cursorObj = con.cursor()
    
    cursorObj.execute("""CREATE TABLE IF NOT EXISTS mapping_traits(
                         experiment text,
                         trait text,
                         description text,
                         plot blob,
                         processed BOOL)
                         """)   

    cursorObj.execute("""CREATE TABLE IF NOT EXISTS mapping_results(
                         experiment text,
                         trait text,
                         snp text,
                         chrom text,
                         pos integer,
                         ref text,
                         alt text,
                         effect REAL,
                         pval REAL)
                         """)
    cursorObj.execute("""CREATE TABLE IF NOT EXISTS mapping_candidates(
                         gene_id text,
                         gene_symbol text,
                         gene_description text, 
                         experiment text,
                         trait text,
                         candidate_count integer,
                         snp text,
                         chrom text,
                         pos integer)
                         """)   
    con.commit()

In [ ]:
# Current implementation requires re-parsing of all the input files to create SQNce
# TODO SQNce update functions to parse input data only if not previously included 
#if os.path.exists("SQNce-proteomes.db"): os.remove("SQNce-proteomes.db")
con = sql_connection()
sql_table(con)
con.close()

# SQNce Mapping-related Functions

In [ ]:
def mapping_traits_insert_traits(con, entities):
    cursorObj = con.cursor()
    cursorObj.executemany("""INSERT INTO mapping_traits(
                         experiment, 
                         trait, 
                         description,
                         plot,
                         processed) 
                         VALUES(?,?,?,?,?)""", entities)
    con.commit()

def mapping_traits_insert_results(con, entities):
    cursorObj = con.cursor()
    cursorObj.executemany("""INSERT INTO mapping_results(
                         experiment, 
                         trait, 
                         snp,
                         chrom,
                         pos,
                         ref,
                         alt,
                         effect,
                         pval) 
                         VALUES(?,?,?,?,?,?,?,?,?)""", entities)
    con.commit()
    
def mapping_traits_insert_candidates(con, entities):
    cursorObj = con.cursor()
    cursorObj.execute("""INSERT INTO mapping_candidates(
                         experiment, 
                         trait, 
                         gene_id,
                         gene_symbol,
                         gene_description,
                         candidate_count,
                         snp,
                         chrom,
                         pos) 
                         VALUES(?,?,?,?,,?,?,?,?,?)""", entities)
    con.commit()



# SQNce Load the rMVP GLM GWAS Results

In [ ]:
con = sql_connection()

# The name of the experiment is also the name of the folder
experiment = "experiment"

fl_pvals_dict = {}
fl_plots_dict = {}
fl_name_set = set()
for fl in os.listdir(experiment):
    # Assume for now only my output files are used
    if fl.split(".")[-1] == "png":
        fl_name = fl.replace(".png", "")
        fl_plots_dict[fl_name] = fl
    if fl.split(".")[-1] == "csv":
        fl_name = fl.replace(".sig.csv", "")
        fl_pvals_dict[fl_name] = fl
    fl_name_set.add(fl_name)
fl_name_set


all_traits_list = []
all_results_list = []

# The name of the trait is also the name of the file
for trait in fl_name_set:
    # Load the trait results dataframe
    filename = os.path.join(experiment, fl_pvals_dict[trait])
    df = pd.read_csv(filename)
    if len(df) == 0:
        continue
    df = df.sort_values(df.columns[-1])

    # Load the trait plot binary variable
    filename = os.path.join(experiment, fl_plots_dict[trait])
    with open(filename, 'rb') as file: plot_blob = file.read()
    
    all_traits_list.append([experiment, trait, "rMVP GLM of 282_SC MR network with E5", plot_blob, False])    

    # List order: experiment,trait,snp,chrom,pos,ref,alt,effect,pval,plot
    #selected_rows_ix = [0]
    #selected_rows = df.iloc[selected_rows_ix,].values.tolist()
    selected_rows = df.values.tolist()
        
    for row in selected_rows:
        all_results_list.append([experiment, trait, row[0], row[1], row[2], row[3], row[4], row[5], row[7]])

mapping_traits_insert_traits(con, all_traits_list)
mapping_traits_insert_results(con, all_results_list)

In [ ]:
from io import BytesIO
def pil_to_b64(im, enc_format="png", **kwargs):
    """
    Converts a PIL Image into base64 string for HTML displaying
    :param im: PIL Image object
    :param enc_format: The image format for displaying. If saved the image will have that extension.
    :return: base64 encoding
    """

    buff = BytesIO()
    im.save(buff, format=enc_format, **kwargs)
    encoded = base64.b64encode(buff.getvalue()).decode("utf-8")

    return encoded

# https://stackoverflow.com/questions/14348442/how-do-i-display-a-pil-image-object-in-a-template#14348661
import PIL
import io
from PIL import Image

picture_stream = io.BytesIO(fig)
picture = Image.open(picture_stream)
# In dash you can send the image to html.Img like this:
# html.Img(id="my-img",className="image", src="data:image/png;base64, " + pil_to_b64(picture))

# picture = PIL.Image.open(picture_stream)
# picture.show()
# pil_to_b64(picture)